In [1]:
import torch

In [92]:
import numpy as np
import os
import numpy as np
import torch
import torch.nn.functional as F
import time
from collections import defaultdict
from torch.optim.lr_scheduler import ExponentialLR
import argparse

os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [93]:
import os
os.path.exists('.')

True

In [94]:
import torch
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from kge.model import KgeModel
from kge.util.io import load_checkpoint


In [126]:
checkpoint = load_checkpoint('../../pretrained_models/embeddings/ComplEx_fbwq_full_new/checkpoint_best.pt')
model = KgeModel.create_from(checkpoint)
model.eval()


Loading configuration of dataset fbwq_full_new...
Setting complex.relation_embedder.dropout to 0, was set to -0.4746062345802784.


ComplEx(
  (_entity_embedder): LookupEmbedder(
    (_embeddings): Embedding(1886683, 256, sparse=True)
    (dropout): Dropout(p=0.44299429655075073, inplace=False)
  )
  (_relation_embedder): LookupEmbedder(
    (_embeddings): Embedding(572, 256, sparse=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (_scorer): ComplExScorer()
)

In [127]:
f = open('../../data/fbwq_full_new/entity_ids.del')
entity2idx = {}
for line in f:
    line = line.strip().split('\t')
    entity2idx[line[1]] = int(line[0])
f.close()

In [171]:
head = 'm.078ffw'
rel = 'book.literary_series.works_in_this_series'
rel_id = 0
rels = torch.Tensor([i for i in range(548)]).long()
for i, s in enumerate(model.dataset.relation_strings(rels)):
    if s == rel:
        rel_id = i
s_id = entity2idx[head]
print(s_id, rel_id)

1255755 449


In [172]:
s = torch.Tensor([s_id]).long()             # subject indexes
p = torch.Tensor([rel_id]).long()             # relation indexes
scores = model.score_sp(s, p)                # scores of all objects for (s,p,?)
sc, o = torch.topk(scores, 20, largest=True, dim=-1)             # index of highest-scoring objects

print(sc)
print(o)
print(model.dataset.entity_strings(s))       # convert indexes to mentions
print(model.dataset.relation_strings(p))
print(model.dataset.entity_strings(o))

x = ComplEx(s_id,rel_id)
y = torch.argmax(x, dim=-1)
y

tensor([[65.0816, 65.0736, 64.3656, 62.1204, 61.6874, 61.4585, 59.6895, 32.7663,
         30.7057, 29.9945, 29.4358, 29.0522, 27.1528, 26.8394, 25.3394, 25.2249,
         24.8479, 24.6993, 24.3568, 24.1571]], grad_fn=<TopkBackward>)
tensor([[  20091,   20093,   31510,   20092,   31509,   20094,   31508,  615194,
           78897,   28577,  955667,   78901,   33591,   59932, 1272405, 1473286,
          246128, 1697097,   28580, 1254688]])
['m.078ffw']
['book.literary_series.works_in_this_series']
[['m.06_rf9' 'm.03bkkv' 'm.015pln' 'm.01lr1g' 'm.0c_vk' 'm.014jst'
  'm.01m5g_' 'm.026jlk9' 'm.03177r' 'm.031778' 'm.015dlb' 'm.03176f'
  'm.03hxsv' 'm.031786' 'm.06hgrx3' 'm.06q7x14' 'm.07699t' 'm.0bqqz1h'
  'm.031hcx' 'm.06f86m6']]


tensor([20091])

In [73]:
def ComplEx(head_id, relation_id):
    head = model._entity_embedder._embeddings(torch.Tensor([head_id]).long())
    relation = model._relation_embedder._embeddings(torch.Tensor([relation_id]).long())
    head = torch.stack(list(torch.chunk(head, 2, dim=1)), dim=1)
    head = head.permute(1, 0, 2)
    re_head = head[0]
    im_head = head[1]
    re_relation, im_relation = torch.chunk(relation, 2, dim=1)
    re_tail, im_tail = torch.chunk(model._entity_embedder._embeddings.weight, 2, dim =1)
    re_score = re_head * re_relation - im_head * im_relation
    im_score = re_head * im_relation + im_head * re_relation
    score = torch.stack([re_score, im_score], dim=1)
    score = score.permute(1, 0, 2)
    re_score = score[0]
    im_score = score[1]
    score = torch.mm(re_score, re_tail.transpose(1,0)) + torch.mm(im_score, im_tail.transpose(1,0))
#     pred = torch.sigmoid(score)
    pred = score
    return pred

In [88]:
r = model._entity_embedder._embeddings(torch.Tensor([91245]).long())
r

tensor([[ 1.9431,  1.8672, -1.2449,  0.4421,  1.6214, -1.9546, -2.1960, -2.1960,
         -0.2738, -0.7340, -1.1670, -0.9711, -1.8178, -0.7259,  0.1633,  1.0926,
          0.5868, -0.2913,  0.8795, -0.2848, -1.2855,  0.7986,  1.8250, -1.2966,
         -1.3353, -0.2483,  2.0538,  0.9398,  1.8204,  0.6461, -0.3098, -1.9697,
          1.2385,  0.7508,  1.9437,  0.6374,  2.9127,  0.1030, -3.1933,  2.9447,
         -1.2377,  0.5684, -0.8346,  0.7918,  0.1616,  1.3663, -1.3035,  1.1497,
         -1.9876, -1.2987,  2.4317,  0.3529, -1.0604,  1.4364,  3.0076, -1.0881,
         -2.1577, -1.4260,  2.1145, -2.5210,  3.3457,  1.2450,  1.8183, -1.9855,
          1.1236,  0.5548,  2.3935, -1.2452, -1.8476,  0.0859,  1.4603,  1.0376,
         -1.4839,  2.3597,  1.2356,  0.9587, -0.5229,  0.6607, -2.2229, -1.6865,
          1.5329,  0.4195,  1.0505, -2.1253, -3.2505,  3.5873,  0.3544,  0.8479,
         -3.2748,  0.3019,  2.3390, -0.9631, -1.9129, -2.9487,  0.3306, -2.2707,
          1.8452, -1.8667, -

In [89]:
x

tensor([[-0.5567,  3.0712, -0.5995,  ..., -6.3965, -2.2072, -5.2597]],
       grad_fn=<AddBackward0>)

In [76]:
scores

tensor([[  1.2696, -15.2891,  -9.8496,  ..., -13.2282,  -9.4417, -11.6062]],
       grad_fn=<ViewBackward>)

In [38]:
entity2idx['m.06w2sn5'] # justin beiber

4890

In [17]:
embedder = model.get_s_embedder()
e = 'm.0chgzm'
e_id = entity2idx[e]
input = torch.LongTensor([e_id])
entity_embedding = embedder._embeddings(input)


In [18]:
entity_embedding[0]

tensor([ 1.0806e+00, -1.3234e+00,  1.1303e+00, -9.0067e-01,  4.3825e-01,
        -3.7421e-01,  1.2426e+00, -1.0830e+00, -9.9529e-01, -1.4535e+00,
        -1.3703e-01,  9.4352e-01, -1.0700e+00,  7.7651e-01,  3.6658e-01,
         2.5738e+00,  7.8383e-02,  4.6301e-02, -5.3422e-01,  2.5738e-01,
        -4.3520e-01, -1.9415e+00,  4.9657e-04, -2.8257e-01, -7.8855e-01,
        -1.4186e+00, -1.0457e+00,  1.3098e+00, -5.1937e-01,  4.0517e-02,
         6.6937e-01, -6.9332e-01,  7.6408e-01, -8.2103e-02, -7.2640e-01,
         3.2012e-01, -7.5737e-01, -1.2666e+00, -6.5312e-01, -7.2035e-01,
        -5.7599e-01, -9.1251e-02, -2.9292e-01,  1.3930e+00, -4.7956e-01,
         2.2868e+00, -5.8547e-01, -1.3357e+00, -8.7579e-01,  1.3007e+00,
        -5.1788e-01,  1.4566e-01,  7.2676e-01,  5.4709e-01, -3.4058e-01,
        -1.5802e+00,  1.3112e+00,  2.1026e-01, -1.6019e+00,  9.4673e-03,
         6.0343e-01,  1.6433e+00, -8.9710e-01, -2.6170e-01,  1.5939e+00,
         8.2037e-01,  3.5385e-01, -5.6484e-02, -4.9

In [16]:
x = model._entity_embedder._embeddings.weight[0]
x

tensor([ 1.0806e+00, -1.3234e+00,  1.1303e+00, -9.0067e-01,  4.3825e-01,
        -3.7421e-01,  1.2426e+00, -1.0830e+00, -9.9529e-01, -1.4535e+00,
        -1.3703e-01,  9.4352e-01, -1.0700e+00,  7.7651e-01,  3.6658e-01,
         2.5738e+00,  7.8383e-02,  4.6301e-02, -5.3422e-01,  2.5738e-01,
        -4.3520e-01, -1.9415e+00,  4.9657e-04, -2.8257e-01, -7.8855e-01,
        -1.4186e+00, -1.0457e+00,  1.3098e+00, -5.1937e-01,  4.0517e-02,
         6.6937e-01, -6.9332e-01,  7.6408e-01, -8.2103e-02, -7.2640e-01,
         3.2012e-01, -7.5737e-01, -1.2666e+00, -6.5312e-01, -7.2035e-01,
        -5.7599e-01, -9.1251e-02, -2.9292e-01,  1.3930e+00, -4.7956e-01,
         2.2868e+00, -5.8547e-01, -1.3357e+00, -8.7579e-01,  1.3007e+00,
        -5.1788e-01,  1.4566e-01,  7.2676e-01,  5.4709e-01, -3.4058e-01,
        -1.5802e+00,  1.3112e+00,  2.1026e-01, -1.6019e+00,  9.4673e-03,
         6.0343e-01,  1.6433e+00, -8.9710e-01, -2.6170e-01,  1.5939e+00,
         8.2037e-01,  3.5385e-01, -5.6484e-02, -4.9